# Explore the initialisation maps
In this tutorial we will explore the initialisation maps offered by `plasticparcels`. Descriptions of these initialisation maps can be found [here](https://plastic.oceanparcels.org/en/latest/initialisationmaps.html)

In [1]:
# Library imports
from datetime import datetime, timedelta
import os
os.chdir('../../')
# Parcels and PlasticParcels imports
import plasticparcels as pp

# Plotting imports
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import pandas as pd

In [2]:
# Load the model settings
settings = {"release_maps": {
                "coastal": None,
                "rivers": None,
                "fisheries": None,
                "global_concentrations": None,},
            "unbeaching": {
                "filename": None,
            }
}

In [3]:
# Download the mask and release data
settings = pp.utils.download_plasticparcels_dataset('NEMO0083', settings, 'input_data')

## Explore the `coastal` initialisation map
We will first load in the data using `pandas`

In [4]:
coastal_ds = pd.read_csv(settings['release_maps']['coastal'])